In [5]:
import numpy as np
import pandas as pd

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import re
import json

In [6]:
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\k.osadchenko\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
sent_analizer = SentimentIntensityAnalyzer()

In [8]:
data_train = pd.read_csv('../input/sf-booking/hotels_train.csv')
data_train.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


In [9]:
data_test = pd.read_csv('../input/sf-booking/hotels_test.csv')
data_test.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,11/26/2015,8.3,Alexandra Barcelona A DoubleTree by Hilton,Sweden,Pillows,3,351,Nice welcoming and service,5,15,"[' Business trip ', ' Solo traveler ', ' Twin ...",616 day,41.393192,2.161520


In [10]:
df_train = data_train.copy()
df_test = data_test.copy()

In [11]:
df = pd.concat([df_train, df_test], axis=0)

In [12]:
positive_review_dict = {}
negative_review_dict = {}

In [14]:
def pos_review_preprocess(msg):
    global positive_review_dict
    
    msg_cleared = msg.strip()
    if msg_cleared:
        positive_review_dict[msg] = sent_analizer.polarity_scores(msg_cleared)
        
def neg_review_preprocess(msg):
    global negative_review_dict
    
    msg_cleared = msg.strip()
    if msg_cleared:
        negative_review_dict[msg] = sent_analizer.polarity_scores(msg_cleared)

In [15]:
df['positive_review'].apply(pos_review_preprocess)
df['negative_review'].apply(neg_review_preprocess)

0         None
1         None
2         None
3         None
4         None
          ... 
128930    None
128931    None
128932    None
128933    None
128934    None
Name: negative_review, Length: 515738, dtype: object

In [16]:
json_object = json.dumps(negative_review_dict, indent=4)
 
with open("../input/sf-booking-sentiment-cahce/negative_review_dict.json", "w") as outfile:
    outfile.write(json_object)

json_object = json.dumps(positive_review_dict, indent=4)
 
with open("../input/sf-booking-sentiment-cahce/positive_review_dict.json", "w") as outfile:
    outfile.write(json_object)

In [17]:
with open('../input/sf-booking-sentiment-cahce/negative_review_dict.json', 'r') as in_file:
    negative_review_dict = json.load(in_file)
    
with open('../input/sf-booking-sentiment-cahce/positive_review_dict.json', 'r') as in_file:
    positive_review_dict = json.load(in_file)

In [ ]:
df['positive_review_neg'] = 0.0
df['positive_review_neu'] = 0.0
df['positive_review_pos'] = 0.0
df['positive_review_compound'] = 0.0

df['negative_review_neg'] = 0.0
df['negative_review_neu'] = 0.0
df['negative_review_pos'] = 0.0
df['negative_review_compound'] = 0.0

In [18]:
df['positive_review_neg'] = df['positive_review'].apply(lambda x: 0.0 if (len(x.strip()) == 0) or (positive_review_dict[x] is None) else positive_review_dict[x]['neg'])
df['positive_review_neu'] = df['positive_review'].apply(lambda x: 0.0 if (len(x.strip()) == 0) or (positive_review_dict[x] is None) else positive_review_dict[x]['neu'])
df['positive_review_pos'] = df['positive_review'].apply(lambda x: 0.0 if (len(x.strip()) == 0) or (positive_review_dict[x] is None) else positive_review_dict[x]['pos'])
df['positive_review_compound'] = df['positive_review'].apply(lambda x: 0.0 if (len(x.strip()) == 0) or (positive_review_dict[x] is None) else positive_review_dict[x]['compound'])

df['negative_review_neg'] = df['negative_review'].apply(lambda x: 0.0 if (len(x.strip()) == 0) or (negative_review_dict[x] is None) else negative_review_dict[x]['neg'])
df['negative_review_neu'] = df['negative_review'].apply(lambda x: 0.0 if (len(x.strip()) == 0) or (negative_review_dict[x] is None) else negative_review_dict[x]['neu'])
df['negative_review_pos'] = df['negative_review'].apply(lambda x: 0.0 if (len(x.strip()) == 0) or (negative_review_dict[x] is None) else negative_review_dict[x]['pos'])
df['negative_review_compound'] = df['negative_review'].apply(lambda x: 0.0 if (len(x.strip()) == 0) or (negative_review_dict[x] is None) else negative_review_dict[x]['compound'])

In [19]:
df.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,...,lat,lng,positive_review_neg,positive_review_neu,positive_review_pos,positive_review_compound,negative_review_neg,negative_review_neu,negative_review_pos,negative_review_compound
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,...,51.507894,-0.143671,0.000,0.345,0.655,0.5859,0.000,1.000,0.0,0.0000
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,...,51.521009,-0.123097,0.000,1.000,0.000,0.0000,0.756,0.244,0.0,-0.4767
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,...,48.845377,2.325643,0.379,0.000,0.621,0.3400,0.423,0.577,0.0,-0.2960
